# Laboratorio 1: Detección de Phishing

In [92]:
#Importar las librerías
import urllib.parse
import pandas_profiling
import pandas as pd
import re
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, r2_score

## Parte 1: Ingeniería de características

In [93]:
## Cargar el dataset proporcionado
df = pd.read_csv('dataset_pishing.csv')
## df.drop(['host', 'subclass'], axis=1, inplace=True)
print(df.shape)
df.sample(n=5).head() # print a random sample of the DataFrame

(11430, 67)


,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
11319,http://67.199.68.195/nosmoke.htm,1,0,0,0,0,0,0,1,1,...,1,0,1,0,-1,0,1,0,0,phishing
10678,http://www.fofweb.com/Electronic_Images/Onfile...,0,1,0,0,0,0,0,0,0,...,1,1,0,26,7644,3871327,0,1,5,legitimate
1706,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,1,0,25,3993,5707171,0,1,0,phishing
3399,http://searchbusinessanalytics.techtarget.com/...,0,0,0,0,0,0,0,0,0,...,1,0,0,134,7615,2188,0,0,5,legitimate
8565,http://cjincjobs.blogspot.com,0,0,0,0,0,0,0,0,0,...,1,1,0,371,7298,334843,0,0,5,legitimate


In [94]:
legit = df.loc[df['status'] == 'legitimate'].shape[0]
phi = df.loc[df['status'] == 'phishing'].shape[0]
print('Phishing ->', phi, '\nLegitimate ->', legit)
print('Proporción de balance para registros legítimos/phishing ->', legit/phi, '\nDataset Balanceado')

Phishing -> 5715 
Legitimate -> 5715
Proporción de balance para registros legítimos/phishing -> 1.0 
Dataset Balanceado


Características basadas en el dominio: <br>

| F1 | URL length |<br>
| F2 | Hostname Length |<br>
| F4 to F20 | Special Characters -> ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', '%20', ' '] |<br>
| F25 | HTTPS token | <br>
| F26 | URL Digits Ratio | <br>
| F27 | URL Digits Ratio Hostname | <br>

In [95]:
# Parsea la url
def parse_url(url):
    return urllib.parse.urlparse(url)

# Calcula la longitud del URL
def url_length(url):
    
    return len(url)

# Calcula la longitud del hostname contenido en la URL
def url_hostname_length(url):
    parsed_url = parse_url(url)
    hostname = parsed_url.hostname
    
    return len(hostname)

# Calcula la cantidad de caracteres especiales contenidos en la url
def count_special_char(url):
    sp_chars = ['.','-','@','?','&','|','=','_','~','%','/','*',':',',',';','$','%20',' ']
    
    return sum(char in sp_chars for char in url)

# Verifica si contiene un protocolo HTTPS
def https_check(url):
    parsed_url = parse_url(url)
    protocol = parsed_url.scheme
    
    return 1 if protocol == "https" else 0

# Calcula el ratio de digitos dentro de la url
def url_digit_ratio(url):
    
    return sum(char.isdigit() for char in url) / len(url)

# Calcula el ratio de digitos del hostname contenido en la url
def hostname_digit_ratio(url):
    parsed_url = parse_url(url)
    hostname = parsed_url.hostname
    
    return sum(char.isdigit() for char in hostname) / len(hostname)

#--------------------------------------------------------------------------------------------------------      

# Pruebas
"""url = 'http://www.inbioma.pe/rechnung-376440790464490488&amp;cgi3-viewkontakt-376440790464490488-007acctpagetype-376440790464490488=4674720-&amp;info@s-und-s-shop.de.html'
print(url_length(url), url_hostname_length(url), count_special_char(url),
      https_check(url), url_digit_ratio(url), hostname_digit_ratio(url))"""

"url = 'http://www.inbioma.pe/rechnung-376440790464490488&amp;cgi3-viewkontakt-376440790464490488-007acctpagetype-376440790464490488=4674720-&amp;info@s-und-s-shop.de.html'\nprint(url_length(url), url_hostname_length(url), count_special_char(url),\n      https_check(url), url_digit_ratio(url), hostname_digit_ratio(url))"

In [96]:
# Aplicación de las funciones construídas
url = df['url']

df['url_length'] = url.apply(url_length)
df['hostname_length'] = url.apply(url_hostname_length)
df['special_chars'] = url.apply(count_special_char)
df['https'] = url.apply(https_check)
df['url_digit_ratio'] = url.apply(url_digit_ratio)
df['hostname_digit_ratio'] = url.apply(hostname_digit_ratio)

df_sampler = df[['url_length','hostname_length','special_chars',
                'https','url_digit_ratio','hostname_digit_ratio','status']]

df_sampler.sample(n=5).head()

,url_length,hostname_length,special_chars,https,url_digit_ratio,hostname_digit_ratio,status
4249,32,17,8,0,0.000000,0.000000,legitimate
5798,31,22,6,1,0.000000,0.000000,legitimate
5007,34,25,6,1,0.029412,0.040000,phishing
11139,37,28,6,1,0.054054,0.071429,phishing
5628,43,35,9,0,0.000000,0.000000,phishing


## Parte 2: Preprocesamiento

In [97]:
# Cambio a variable categorica
df_sampler['status'] = df_sampler['status'].map({'phishing': 1, 'legitimate': 0})
print(df_sampler['status'])

feature_names = df_sampler

0        0
1        1
2        1
3        0
4        0
        ..
11425    0
11426    1
11427    0
11428    0
11429    1
Name: status, Length: 11430, dtype: int64


C:\Users\olive\AppData\Local\Temp\ipykernel_32384\192044610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sampler['status'] = df_sampler['status'].map({'phishing': 1, 'legitimate': 0})


## Visualización de la data

In [12]:
"""profile = pandas_profiling.ProfileReport(df)
profile.to_file("output.html")"""

'profile = pandas_profiling.ProfileReport(df)\nprofile.to_file("output.html")'

## Parte 2: Implementación

In [112]:
# Separación de la data de entrenamiento, prueba y validación
data_remain, val_data = model_selection.train_test_split(feature_names, test_size=0.15, random_state=42)

# validation data is storaged
val_data.to_csv('val_data_phi.csv', index=False)
# Validation Data - 15% of total data
y_val = val_data['status']
x_val = val_data.drop(['status'], axis=1)

# 85% of remaining data
target = data_remain['status']
X = data_remain.drop(['status'], axis=1)

# Training Data 55% - Testing Data 30%
x_train, x_test, y_train,y_test = model_selection.train_test_split(X,target, test_size=0.30/(0.55 + 0.30), random_state=42)

# training data is storaged
train_data = x_train.copy()
train_data['status'] = y_train
train_data.to_csv('train_data_phi.csv', index=False)

# testing data is storaged
test_data = x_test.copy()
test_data['status'] = y_test
test_data.to_csv('test_data_phi.csv', index=False)

In [113]:
x_train.count()

url_length              6286
hostname_length         6286
special_chars           6286
https                   6286
url_digit_ratio         6286
hostname_digit_ratio    6286
dtype: int64

In [114]:
x_test.count()

url_length              3429
hostname_length         3429
special_chars           3429
https                   3429
url_digit_ratio         3429
hostname_digit_ratio    3429
dtype: int64

In [115]:
val_data.count()

url_length              1715
hostname_length         1715
special_chars           1715
https                   1715
url_digit_ratio         1715
hostname_digit_ratio    1715
status                  1715
dtype: int64

In [116]:
print(y_train.value_counts())
print(y_test.value_counts())
print(y_val.value_counts())

1    3158
0    3128
Name: status, dtype: int64
0    1716
1    1713
Name: status, dtype: int64
0    871
1    844
Name: status, dtype: int64


## Entrenamiento del modelo

### Linear Regression

In [140]:
regressor = LinearRegression()
regressor.fit(x_train, y_train)

y_test_pred = regressor.predict(x_test)
y_val_pred = regressor.predict(x_val)

# Métricas para testing
print('Testing Data Accuracy: ', metrics.accuracy_score(y_test, y_test_pred.round()))
test_m = metrics.confusion_matrix(y_test, y_test_pred.round())
print('\nTesting Confusion Matrix:\n\n------------------------\n', test_m ,'\n------------------------\n')
print('True Positive: ', test_m[0][0], '[Se predijo phishing y fue phishing]')
print('False Positive: ', test_m[0][1], '[Se predijo phishing y fue legítimo]')
print('False Negative: ', test_m[1][0], '[Se predijo legítimo y fue phishing]')
print('True Negative: ', test_m[1][1],'[Se predijo legítimo y fue legítimo]\n')
print(metrics.classification_report(y_test, y_test_pred.round(), target_names=['legit', 'phishing', '???']))
print('\n---------------------------------------------------------------------------')

# Métricas para validation
print('Validation Data Accuracy: ',metrics.accuracy_score(y_val, y_val_pred.round()))
val_m = metrics.confusion_matrix(y_val, y_val_pred.round())
print('\Validation Confusion Matrix:\n\n------------------------\n', val_m ,'\n------------------------\n')
print('True Positive: ', val_m[0][0], '[Se predijo phishing y fue phishing]')
print('False Positive: ', val_m[0][1], '[Se predijo phishing y fue legítimo]')
print('False Negative: ', val_m[1][0], '[Se predijo legítimo y fue phishing]')
print('True Negative: ', val_m[1][1],'[Se predijo legítimo y fue legítimo]\n')
print(metrics.classification_report(y_val, y_val_pred.round(), target_names=['legit', 'phishing','???']))


Testing Data Accuracy:  0.663167104111986

Testing Confusion Matrix:

------------------------
 [[1409  307    0]
 [ 844  865    4]
 [   0    0    0]] 
------------------------

True Positive:  1409 [Se predijo phishing y fue phishing]
False Positive:  307 [Se predijo phishing y fue legítimo]
False Negative:  844 [Se predijo legítimo y fue phishing]
True Negative:  865 [Se predijo legítimo y fue legítimo]

              precision    recall  f1-score   support

       legit       0.63      0.82      0.71      1716
    phishing       0.74      0.50      0.60      1713
         ???       0.00      0.00      0.00         0

    accuracy                           0.66      3429
   macro avg       0.45      0.44      0.44      3429
weighted avg       0.68      0.66      0.65      3429


---------------------------------------------------------------------------
Validation Data Accuracy:  0.673469387755102
\Validation Confusion Matrix:

------------------------
 [[729 142   0]
 [417 426   1]


C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetri

### Decision Tree

In [137]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)

target_pred = clf.predict(x_test)
val_pred = clf.predict(x_val)

# Métricas para testing
print('Testing Data Accuracy: ', metrics.accuracy_score(y_test, target_pred))
test_m = metrics.confusion_matrix(y_test, target_pred)
print('\nTesting Confusion Matrix:\n\n------------------------\n', test_m ,'\n------------------------\n')
print('True Positive: ', test_m[0][0], '[Se predijo phishing y fue phishing]')
print('False Positive: ', test_m[0][1], '[Se predijo phishing y fue legítimo]')
print('False Negative: ', test_m[1][0], '[Se predijo legítimo y fue phishing]')
print('True Negative: ', test_m[1][1],'[Se predijo legítimo y fue legítimo]\n')
print(metrics.classification_report(y_test, target_pred, target_names=['legit', 'phishing']))
print('\n---------------------------------------------------------------------------')

# Métricas para validation
print('Validation Data Accuracy: ', metrics.accuracy_score(y_val, val_pred))
val_m = metrics.confusion_matrix(y_val, val_pred)
print('\nTesting Confusion Matrix:\n\n------------------------\n', val_m ,'\n------------------------\n')
print('True Positive: ', val_m[0][0], '[Se predijo phishing y fue phishing]')
print('False Positive: ', val_m[0][1], '[Se predijo phishing y fue legítimo]')
print('False Negative: ', val_m[1][0], '[Se predijo legítimo y fue phishing]')
print('True Negative: ', val_m[1][1],'[Se predijo legítimo y fue legítimo]\n')
print(metrics.classification_report(y_val, val_pred, target_names=['legit', 'phishing']))


Testing Data Accuracy:  0.7258675998833479

Testing Confusion Matrix:

------------------------
 [[1294  422]
 [ 518 1195]] 
------------------------

True Positive:  1294 [Se predijo phishing y fue phishing]
False Positive:  422 [Se predijo phishing y fue legítimo]
False Negative:  518 [Se predijo legítimo y fue phishing]
True Negative:  1195 [Se predijo legítimo y fue legítimo]

              precision    recall  f1-score   support

       legit       0.71      0.75      0.73      1716
    phishing       0.74      0.70      0.72      1713

    accuracy                           0.73      3429
   macro avg       0.73      0.73      0.73      3429
weighted avg       0.73      0.73      0.73      3429


---------------------------------------------------------------------------
Validation Data Accuracy:  0.7236151603498542

Testing Confusion Matrix:

------------------------
 [[658 213]
 [261 583]] 
------------------------

True Positive:  658 [Se predijo phishing y fue phishing]
False

## DISCUSIÓN

1. **¿Cuál es el impacto de clasificar un sitio legítimo como Pishing?**

Afectaría directamente las operaciones del sitio legítimo, provocando perdidas de tráfico con posibilidad de negocio y rentabilidad, perjuicios en la confianza pública del website y posibles problemas legales por los daños/perjuicios producidos por su cese de operaciones.

2. **¿Cuál es el impacto de clasificar un sitio de Pishing como legítimo?**

En primera instancia, propiciaría la ejecución de ataques basados en el engaño de usuarios, posibilitando el acceso a información confidencial (contraseñas, información bancaria, información sensible, etc.). Además, podría erosionar la confianza de las entidades tanto clasificadoras como vulneradas, provocando perdidas económicas y una ampliación en el alcance del ataque.

3. **En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares de clasificación de pishing?**

Escogería la precisión del modelo cómo métrica principal, esto con el fin de determinar el ratio de predicciones exitosas en la identificación. Además, elegiría la matriz de confusión para determinar la distribución de predicciones y determinar el modelo que menos daños pueda provocar.

4. **¿Qué modelo es mejor para la clasificación de Pishing? Justifique**

Para poder determinar el mejor modelo, hemos de exponer sus resultados:

**Regresión Lineal:** Este modelo presentó una precisión promedio del 66.5% para los datos de testeo y validación, implicando que 2 de 3 predicciones serán correctas. Al consultar sus matrices de confusión, se observó que aproximadamente el 8.55% de las predicciones, clasificaran sitios legítimos como phishing y un ~24.45% identificará phishing como legítimo.

**Árbol de Decisión:** El modelo retornó una precisión promedio del 72.4% entre los datasets de testeo y validación. Su matriz de confusión indicó un ~12.35% de predicciones falsas positivas y ~15.15% para predicciones falsas negativas.

Con base a los resultados consultados, podemos determinar que el árbol de decisión tiene una mayor precisión identificando phishing (72.6% > 66.5%). Por otra parte, el modelo de regresión lineal presenta un -4.65% de predicciones falsas positivas a comparación de su modelo contraparte, lo que indicaría una mejor métrica en cuanto a la mitigación de perdidas de tráfico y reputación provocada por malas predicciones. Finalmente, el árbol de decisión presentó un -9.3% en predicciones falsas negativas, lo que implica una evasión aumentada de ataques por phishing. Convirtiendolo en un modelo superior en cuanto a la detección de phishing.

Por lo que el árbol de decisión comprende un mejor modelo de clasificación para este dataset.

5. **En base a las métricas obtenidas ¿es necesaria la intervención de una persona humana para tomar la decisión final?**

Dado que ambos modelos cuentan con metricas de precisión insuficientes, la intervención humana será un recurso demandado (y costoso) para la determinación de decisiones finales con la mayor cantidad de mitigación de daño por imprecisión. 